<a href="https://colab.research.google.com/github/Nusserle/LieDetection/blob/master/LieDetectionTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd drive/My Drive/Shared/BagOfLies/


/content/drive/My Drive/Shared/BagOfLies


In [4]:
%cd BagOfLies

[Errno 2] No such file or directory: 'BagOfLies'
/content/drive/My Drive/Shared/BagOfLies


In [0]:
df = pd.read_csv("Annotations.csv")

In [6]:
df.head()

,eeg,gaze,image,run,truth,usernum,video
0,./Finalised/User_0/run_0/EEG.csv,./Finalised/User_0/run_0/Gaze.csv,./Finalised/User_0/run_0/image.csv,0,1,0,./Finalised/User_0/run_0/video.mp4
1,./Finalised/User_0/run_1/EEG.csv,./Finalised/User_0/run_1/Gaze.csv,./Finalised/User_0/run_1/image.csv,1,0,0,./Finalised/User_0/run_1/video.mp4
2,./Finalised/User_0/run_2/EEG.csv,./Finalised/User_0/run_2/Gaze.csv,./Finalised/User_0/run_2/image.csv,2,0,0,./Finalised/User_0/run_2/video.mp4
3,./Finalised/User_0/run_3/EEG.csv,./Finalised/User_0/run_3/Gaze.csv,./Finalised/User_0/run_3/image.csv,3,0,0,./Finalised/User_0/run_3/video.mp4
4,./Finalised/User_0/run_4/EEG.csv,./Finalised/User_0/run_4/Gaze.csv,./Finalised/User_0/run_4/image.csv,4,0,0,./Finalised/User_0/run_4/video.mp4


In [0]:
df_eeg = df[['eeg', 'truth']].dropna().reset_index()

In [8]:
data = []
current_max_dimension = 0
for i, eeg in df_eeg.iterrows():
  df_curr = pd.read_csv(eeg[1]).iloc[:,1:-1]
  if df_curr.shape[0] > current_max_dimension:
    current_max_dimension = df_curr.shape[0]
print(current_max_dimension)

data = np.zeros((df_eeg.shape[0], current_max_dimension, 28))
label = []
for i, eeg in df_eeg.iterrows():
  df_cleared = df_curr.apply(lambda x: pd.to_numeric(x, errors = 'coerce')).dropna()
  x = df_cleared.values #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  for k, row in enumerate(x_scaled):
    for j, val in enumerate(row):
      data[i][k][j] = val
  label.append(eeg[2])


KeyboardInterrupt: ignored

In [0]:
label = np.asarray(label)

In [0]:
print(data.shape)
print(label.shape)

In [0]:
X_train = data[:120]
y_train = label[:120]
X_val = data[120:150]
y_val = label[120:150]
X_test = data[150:]
y_test = data[150:]

In [0]:
model = models.Sequential()
# model.add(layers.LSTM(100, activation='tanh' ,input_shape=(4867,28,)))
model.add(layers.Dense(4869, activation='relu' ,input_shape=(4867,28,)))
model.add(layers.Flatten())
model.add(layers.Dense(2000, activation='relu'))
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(X_train,
                    y_train,
                    epochs=5,
                    batch_size=30,
                    validation_data=(X_val, y_val), verbose=1)

In [0]:
model2 = models.Sequential()
#model2.add(layers.LSTM(57, activation='tanh' ,input_shape=(57,300,)))
model2.add(layers.Dense(57, activation='relu' ,input_shape=(57,300,)))
#model.add(layers.Conv1D(64, 5, activation='relu',input_shape=(57,300,)))
#model.add(layers.GlobalMaxPooling1D())
#model.add(layers.GRU(64, activation='tanh', input_shape=(57,300,)))
model2.add(layers.Flatten())
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(38, activation='relu'))
model2.add(layers.Dropout(0.2))
model2.add(layers.Dense(16, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['acc'])